In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# import imageio as io
import imageio.v2 as imageio
import os
from sklearn.decomposition import PCA
import skimage
from skimage import color
from skimage import io

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#ubicacion de las carpetas con cada tipo de flor
path = "/content/drive/MyDrive/Datas/data_flower/"

path_amarilla = "f_amarilla/"
path_blanca = "f_blanca/"
path_morada = "f_morada/"
path_rosa = "f_rosa/"

imgs_amarilla = os.listdir(path + path_amarilla)  # 41
imgs_blanca = os.listdir(path + path_blanca)  # 57
imgs_morada = os.listdir(path + path_morada)  # 39
imgs_rosa = os.listdir(path + path_rosa)  # 39

imgtest = imageio.imread(path + 'f_' + 'amarilla' + '/' + imgs_amarilla[0])


In [4]:
#Lectura de imagenes, redimensionamiento de matriz a vector y concatenacion
def lst_vector_color(archivos_color, nombre_color, id_color):
    path_color = 'f_' + nombre_color + '/'
    num_flat = 128 * 128 * 4
    lst_vectores = []
    lst_class = []
    for i in range(0, len(archivos_color)):
        img = imageio.imread(path + path_color + archivos_color[i])
        # img = io.imread(path + path_color + archivos_color[i] , as_gray=True)
        img_rshp = img.reshape(1, num_flat)[0]
        lst_vectores.append(img_rshp)
        array_vectores = np.array(lst_vectores)
        lst_class.append(id_color)
    return array_vectores, lst_class

In [5]:
array_vect_amarilla, id_amarilla = lst_vector_color(imgs_amarilla, 'amarilla', 0)
array_vect_blanca, id_blanca = lst_vector_color(imgs_blanca, 'blanca', 1)
array_vect_morada, id_morada = lst_vector_color(imgs_morada, 'morada', 2)
array_vect_rosa, id_rosa = lst_vector_color(imgs_rosa, 'rosa', 3)

In [6]:
# Ordenamos  la matriz de caracteristicas
x = np.concatenate((array_vect_amarilla, array_vect_blanca, array_vect_morada, array_vect_rosa),
                   axis=0)  # (160, 65536) n>p no se cumple, se tiene que reducir
y = np.array(id_amarilla + id_blanca + id_morada + id_rosa).reshape(160, 1)
xy = np.concatenate((x, y), axis=1)
x_std = (x - x.mean()) / (x.std())
n1, p1 = list(x_std.shape)[0], list(x_std.shape)[1]

test1 = pd.DataFrame(xy).add_prefix('pixel_').rename(columns={'pixel_65536': 'Clase'})

In [7]:
# the n_components of PCA must be lower than min(n_samples, n_features).
pca = PCA(n_components=60)
pca.fit(x_std)
X_proj = pca.fit_transform(x_std)
loading = pd.DataFrame(pca.components_.T).round(2)
componentes = pca.components_ #eigenvector (60, 65536)
componentes = pd.DataFrame(data= componentes,columns = list(test1.columns)[:-1]).round(2)
print(pca.explained_variance_ratio_)
print(np.cumsum(pca.explained_variance_ratio_))
eigen_valores = pd.DataFrame(pca.explained_variance_).round(2)
ratio_var_exp = pd.DataFrame(pca.explained_variance_ratio_)
cum_var_exp = pd.DataFrame( np.cumsum(pca.explained_variance_ratio_))
var_cum_normal = pd.concat([ratio_var_exp,cum_var_exp], axis = 1)




[0.15830808 0.07886639 0.04398274 0.03900211 0.03077903 0.02714626
 0.02279404 0.0214917  0.01968181 0.01881374 0.01504145 0.01394624
 0.01271494 0.01252147 0.01195665 0.01144749 0.01097819 0.01044168
 0.00953666 0.00912447 0.00902014 0.0088513  0.00857502 0.00850325
 0.00819504 0.00770671 0.00760477 0.00721075 0.00686606 0.00675525
 0.00667239 0.00634321 0.00621344 0.00612794 0.00588579 0.0057656
 0.00554252 0.00534307 0.00525139 0.00518406 0.0050806  0.00495018
 0.00484519 0.00472457 0.00454987 0.00448229 0.00445368 0.00437504
 0.00430333 0.0042514  0.00420412 0.00413589 0.00395    0.00389467
 0.00387585 0.00377235 0.0037466  0.00364059 0.00356712 0.00348872]
[0.15830808 0.23717446 0.2811572  0.32015931 0.35093835 0.37808461
 0.40087865 0.42237035 0.44205216 0.4608659  0.47590736 0.48985359
 0.50256853 0.51509    0.52704664 0.53849413 0.54947232 0.559914
 0.56945066 0.57857512 0.58759526 0.59644656 0.60502158 0.61352483
 0.62171987 0.62942658 0.63703135 0.6442421  0.65110816 0.657863

In [8]:
#se calcula la inversa del pca
X_inv_proj = pca.inverse_transform(X_proj)
#se redimensiona
X_proj_img = np.reshape(X_inv_proj, (160, 128, 128, 4))
#SE elige una imagen
img1_recov = X_proj_img[121]
#Se des-estandariza
x_1 = (img1_recov * x.std()) + x.mean()

In [9]:
# desriptivos
x_pd_grp = pd.DataFrame(xy).groupby(128 * 128 * 4).max()
x_pd_grp.round(2)

,0,1,2,3,4,5,6,7,8,9,...,65526,65527,65528,65529,65530,65531,65532,65533,65534,65535
65536,,,,,,,,,,,,,,,,,,,,,
0,183,189,208,255,229,190,209,255,195,193,...,255,255,255,255,255,255,255,255,255,255
1,237,240,235,255,150,190,139,255,151,192,...,104,255,113,106,91,255,217,217,217,255
2,193,209,213,255,196,205,213,255,204,209,...,179,255,199,231,186,255,203,229,188,255
3,253,209,227,255,253,209,214,255,253,209,...,210,255,248,211,211,255,229,217,213,255


In [11]:
X_proj.shape

(160, 60)

In [13]:
#y son las labels de las flores

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_proj, y, test_size=0.2, random_state=42)


from sklearn.neighbors import NearestNeighbors
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

print("Predictions from the classifier:")
test_data_predicted = knn.predict(X_test)
print(test_data_predicted)
print("Target values:")
print(test_labels)

NameError: ignored